In [ ]:
# -*- coding: utf-8 -*-

"""Contains function to perform GIM3E"""

from __future__ import absolute_import

import six
from optlang.symbolics import Zero
from cobra.util import fix_objective_as_constraint

def gim3e(model, expression_profile, fraction_of_optimum=0.9, fraction_of_penalty=1.01, condition=None):
    """
    Gene Inactivation Moderated by Metabolism, Metabolomics and Expression[1]

    Parameters
    ----------
    model: cobra.Model
        A constraint-based model to perform GIMME on.
    expression_profile: ExpressionProfile
        An expression profile to integrate in the model.
    fraction_of_optimum: float, optional (default 0.9)
        The fraction of the Required Metabolic Functionalities.
    fraction_of_penalty: float, optional (default 1.01)
        The fraction of the penalty bound.
    condition: str or int, optional (default None)
        The condition from the expression profile. If None, the first condition is used.

    Returns
    -------
    context-specfic model: cobra.Model
    sol: float

    References:
    ----------
    .. [1] Brian J. Schmidt, Ali Ebrahim, Thomas O. Metz, Joshua N. Adkins, Bernhard Ø. Palsson, Daniel R. Hyduke;
           GIM3E: condition-specific models of cellular metabolism developed from metabolomics and expression data,
           Bioinformatics, Volume 29, Issue 22, 15 November 2013, Pages 2900–2908,
           https://doi.org/10.1093/bioinformatics/btt493
    """
    assert isinstance(model, cobra.Model)
    assert isinstance(expression_profile, ExpressionProfile)
    assert isinstance(fraction_of_optimum, float)
    assert isinstance(condition, (str, int, None))
    
    # TODO:
    exp_max = # max corrected log2 intensity
    
    with model:
        prob = model.problem
        condition = expression_profile.conditions[0] if condition is None else condition
        reaction_profile = expression_profile.to_reaction_dict(condition, model, not_measured_value, normalization)
        # Step 1: FBA
        model.objective.direction = "max"
        o_opt = model.slim_optimize()
        fix_objective_as_constraint(model, fraction=fraction_of_optimum)
        # Step 2: Add turnover metabolites
        # 2.1 Make model irreversible
        for rxn in model.reactions:
            if rxn.reversibility is True:
                reverse_rxn = rxn.copy()
                # Fix names
                reverse_rxn.name = rxn.name + " reverse"
                rxn.name += " forward"
                # Fix IDs
                reverse_rxn.id = rxn.id + "_reverse"
                rxn.id += "_forward"
                # Fix bounds
                reverse_rxn.bounds = (0, -rxn.lower_bound)
                rxn.bounds = (0, rxn.upper_bound)
                # Fix metabolite coefficients
                reverse_rxn.subtract_metabolites(reverse_rxn.metabolites, combine=False)
                # Add the reverse reaction to the model
                model.add_reaction(reverse_rxn)
        # 2.2 Turnover metabolites
        turnover_mets = []
        for met in model.metabolites:
            turnover_met = met.copy()
            turnover_met.name = met.name + " turnover"
            turnover_met.id = met.id + "_turnover"
            turnover_mets.append(turnover_met)
        model.add_metabolites(turnover_mets)
        # 2.3 Add turnover metabolites to reactions
        for rxn in model.reactions:
            mets_to_add = {}
            for key, value in six.iteritems(rxn.metabolites):
                turnover_met = model.metabolites.get_by_id(key.id + "_turnover")
                mets_to_add[turnover_met] = abs(value)
            rxn.add_metabolites(mets_to_add)
        # 2.4 Sink reactions
        for met in model.metabolites.query("_turnover$""):
            sink_rxn = cobra.Reaction("TMS_" + met.id)
            # Since both creation and consumption of
            # the real metabolite drive the turnover,
            # we require 2 units of metabolite per
            # 1 unit of flux sink so this matches
            # the turnover through the real metabolite.
            sink_rxn.add_metabolites({met: -2})
            sink_rxn.lower_bound = 1.01 * model.solver.configuration.tolerances.feasibility        
            irr_model.add_reaction(sink_rxn)
        # Step 3: Add penalty coefficients
        penalty_coefficients = {rxn_id: exp_max - rxn_exp for rxn_id, rxn_exp in six.iteritems(reaction_profile)}
        # Step 4: Determine penalty bounds
        penalty_obj_vars = []
        for rxn_id, coefficient in six.iteritems(penalty_coefficients):
            rxn = model.reactions.get_by_id(rxn_id)
            penalty_obj_vars.append(coefficient * rxn.flux_expression)

        model.objective = prob.Objective(Zero, sloppy=True, direction="min")
        model.objective.set_linear_coefficients({v: 1.0 for v in penalty_obj_vars})
        penalty_obj_value = model.slim_optimize()
        penalty_bound_const = prob.Constraint(model.objective.expression,
                                              name="penalty_objective_bound",
                                              ub= fraction_of_penalty * penalty_obj_value)
        model.add_cons_vars(penalty_bound_const)
        # Step 5: Conversion to MILP
        obj_vars = []
        consts = []
        for rxn in model.reactions.query("_forward$"):
            var = prob.Objective(rxn.id + "binary_var_forward",
                                 type="binary")
            const = prob.Constraint(((1 - var) * rxn.upper_bound) - rxn.flux_expression,
                                    lb=0.0, name=rxn.id + "binary_const_forward")
            obj_vars.append(var)
            consts.extend([var, const])
        for rxn in model.reactions.query("_reverse$"):
            var = prob.Objective(rxn.id + "binary_var_reverse",
                                 type="binary")
            const = prob.Constraint((var * rxn.upper_bound) - rxn.flux_expression,
                                    lb=0.0, name=rxn.id + "binary_const_reverse")
            obj_vars.append(var)
            consts.extend([var, const])
        
        model.add_cons_vars(consts)
        sol = model.slim_optimize()

        return (model, sol)